<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/T5_F3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar dependencias
!pip install transformers pandas torch scikit-learn bert-score nltk tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
# Importar librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
import math
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score as bert_score
from tabulate import tabulate

In [ ]:
# Configuración de hiperparámetros iniciales
HYPERPARAMS = {
    "model_name": "t5-small",
    "max_length": 128,
    "batch_size": 8, #parámetro seleccionado con base en la optimización bayesiana.
    "num_epochs": 14, #parámetro seleccionado con base en la optimización bayesiana.
    "learning_rate": 5.286122063435074e-05, #parámetro seleccionado con base en la optimización bayesiana.
    "num_beams": 4, #parámetro seleccionado con base en la optimización bayesiana.
    "no_repeat_ngram_size": 3, #parámetro seleccionado con base en la optimización bayesiana.
    "early_stopping": True,
    "logging_steps": 100,
    "output_dir": "./noticias_t5_model",
    "logging_dir": "./logs",
    "save_steps": 500,
    "save_total_limit": 2
}

In [ ]:
# Cargar datos
from google.colab import files
uploaded = files.upload()

data = pd.read_excel('BASE_30122024.xlsx')
data = pd.DataFrame(data)

Saving BASE_30122024.xlsx to BASE_30122024.xlsx


In [ ]:
# Convertir datos numéricos a texto
def datos_a_texto(row):
    texto_datos = (
        f"Año: {row.get('Año', 'NaN')}, Mes: {row.get('Mes', 'NaN')}, "
        f"Trimestre: {row.get('Trimestre', 'NaN')}, Frecuencia: {row.get('Frecuencia', 'NaN')}, "
        f"Año_comparación: {row.get('Año_comparación', 'NaN')}, País: {row.get('País', 'NaN')}, "
        f"Tendencia: {row.get('Tendencia', 'NaN')}, Sector: {row.get('Sector', 'NaN')}, "
        f"Indicador: {row.get('Indicador', 'NaN')}, Valor_actual: {row.get('Valor_actual', 'NaN')}, "
        f"Variación: {row.get('Variación', 'NaN')}, Valor_comparación: {row.get('Valor_comparación', 'NaN')}, "
        f"Entidad: {row.get('Entidad', 'NaN')}"
    )
    return texto_datos

data["entrada_texto"] = data.apply(datos_a_texto, axis=1)
data["texto_final"] = data["entrada_texto"] + " </s> Noticia: " + data["Noticia"]

In [ ]:
# Clase Dataset personalizada
class NoticiasDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entrada_texto = self.data.iloc[idx]["entrada_texto"]
        noticia = self.data.iloc[idx]["Noticia"]

        input_encoding = self.tokenizer(
            entrada_texto,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            noticia,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": target_encoding["input_ids"].flatten(),
        }

In [ ]:
# Dividir datos en 80% entrenamiento y 20% evaluación
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Cargar tokenizador y modelo preentrenado
tokenizer = T5Tokenizer.from_pretrained(HYPERPARAMS["model_name"])
model = T5ForConditionalGeneration.from_pretrained(HYPERPARAMS["model_name"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Crear datasets
train_dataset = NoticiasDataset(train_data, tokenizer, HYPERPARAMS["max_length"])
eval_dataset = NoticiasDataset(eval_data, tokenizer, HYPERPARAMS["max_length"])

In [ ]:
# Crear DataLoader para los conjuntos de datos
train_dataloader = DataLoader(train_dataset, batch_size=HYPERPARAMS["batch_size"], shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=HYPERPARAMS["batch_size"], shuffle=False)

In [ ]:
# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir=HYPERPARAMS["output_dir"],
    num_train_epochs=HYPERPARAMS["num_epochs"],
    per_device_train_batch_size=HYPERPARAMS["batch_size"],
    save_steps=HYPERPARAMS["save_steps"],
    save_total_limit=HYPERPARAMS["save_total_limit"],
    logging_dir=HYPERPARAMS["logging_dir"],
    logging_steps=HYPERPARAMS["logging_steps"],
    learning_rate=HYPERPARAMS["learning_rate"],
)

# Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Entrenar modelo
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,2.348300
200,0.756300
300,0.632900
400,0.543100
500,0.491800
600,0.441600
700,0.417100
800,0.406800
900,0.360600
1000,0.368600


TrainOutput(global_step=4466, training_loss=0.32937005662725904, metrics={'train_runtime': 805.1056, 'train_samples_per_second': 44.342, 'train_steps_per_second': 5.547, 'total_flos': 1207925578137600.0, 'train_loss': 0.32937005662725904, 'epoch': 14.0})

In [ ]:
# Función para generar texto con parámetros específicos
def generar_noticia_con_parametros(model, tokenizer, datos, temperature, top_p):
    entrada_texto = datos_a_texto(datos)
    inputs = tokenizer.encode(entrada_texto, return_tensors="pt").to(model.device)

    outputs = model.generate(
        inputs,
        max_length=HYPERPARAMS["max_length"],
        temperature=temperature,
        top_p=top_p,
        do_sample=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# Métricas de evaluación
def calcular_perplejidad(model, tokenizer, eval_dataloader):
    model.eval()
    total_loss = 0
    total_tokens = 0

    for batch in eval_dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item() * input_ids.size(0)
            total_tokens += input_ids.size(0)

    return math.exp(total_loss / total_tokens)

def evaluar_parametros(model, tokenizer, eval_data, temperature, top_p):
    generadas = []
    referencias = []

    for i in range(len(eval_data)):
        datos = eval_data.iloc[i]
        referencias.append(datos["Noticia"])
        generadas.append(generar_noticia_con_parametros(model, tokenizer, datos, temperature, top_p))

    P, R, F1 = bert_score(generadas, referencias, lang="es", model_type="bert-base-uncased")
    bleu_scores = [
        sentence_bleu([ref.split()], gen.split()) for ref, gen in zip(referencias, generadas)
    ]
    perplejidad = calcular_perplejidad(model, tokenizer, eval_dataloader)

    return {
        "temperature": temperature,
        "top_p": top_p,
        "bert_f1": F1.mean().item(),
        "bert_Precision": P.mean().item(),
        "bert_Recall": R.mean().item(),
        "bleu": np.mean(bleu_scores),
        "perplejidad": perplejidad
    }

In [ ]:
# Rango de valores para temperature y top_p
temperatures = [0.5, 0.7, 1.0, 1.2, 1.5]
top_p_values = [0.5, 0.6, 0.7, 0.8, 0.9]

# Evaluar todas las combinaciones y guardar los resultados
resultados = []
for temp in temperatures:
    for tp in top_p_values:
        print(f"Evaluando temperature={temp}, top_p={tp}...")
        resultado = evaluar_parametros(model, tokenizer, eval_data, temp, tp)
        resultados.append(resultado)

# Crear la tabla de resultados
tabla_resultados = pd.DataFrame(resultados)
print("\nResultados completos:")
print(tabulate(tabla_resultados, headers="keys", tablefmt="pretty"))

# Seleccionar la mejor combinación
mejor_combinacion = min(
    resultados,
    key=lambda x: (-x["bert_f1"], -x["bleu"], x["perplejidad"])
)

# Mostrar la mejor combinación
print("\nMejor combinación encontrada:")
print(f"Temperature: {mejor_combinacion['temperature']}")
print(f"Top_p: {mejor_combinacion['top_p']}")
print(f"BERT F1: {mejor_combinacion['bert_f1']:.2f}")
print(f"BERT Precision: {mejor_combinacion['bert_Precision']:.2f}")
print(f"BERT Recall: {mejor_combinacion['bert_Recall']:.2f}")
print(f"BLEU: {mejor_combinacion['bleu']:.2f}")
print(f"Perplejidad: {mejor_combinacion['perplejidad']:.2f}")

Evaluando temperature=0.5, top_p=0.5...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Evaluando temperature=0.5, top_p=0.6...
Evaluando temperature=0.5, top_p=0.7...
Evaluando temperature=0.5, top_p=0.8...
Evaluando temperature=0.5, top_p=0.9...
Evaluando temperature=0.7, top_p=0.5...
Evaluando temperature=0.7, top_p=0.6...
Evaluando temperature=0.7, top_p=0.7...
Evaluando temperature=0.7, top_p=0.8...
Evaluando temperature=0.7, top_p=0.9...
Evaluando temperature=1.0, top_p=0.5...
Evaluando temperature=1.0, top_p=0.6...
Evaluando temperature=1.0, top_p=0.7...
Evaluando temperature=1.0, top_p=0.8...
Evaluando temperature=1.0, top_p=0.9...
Evaluando temperature=1.2, top_p=0.5...
Evaluando temperature=1.2, top_p=0.6...
Evaluando temperature=1.2, top_p=0.7...
Evaluando temperature=1.2, top_p=0.8...
Evaluando temperature=1.2, top_p=0.9...
Evaluando temperature=1.5, top_p=0.5...
Evaluando temperature=1.5, top_p=0.6...
Evaluando temperature=1.5, top_p=0.7...
Evaluando temperature=1.5, top_p=0.8...
Evaluando temperature=1.5, top_p=0.9...

Resultados completos:
+----+-----------